<a href="https://colab.research.google.com/github/EmmanuelABonsu/CE888/blob/main/src/lab3/bank_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>